In [5]:
#importing useful libraries

import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
import itertools
import matplotlib.colors as mcolors
import io
from PIL import Image
import gc
import os
import re
import json
import imageio
import time
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
from scipy.ndimage import rotate
from scipy.optimize import minimize


In [6]:
#some  global variables

MAX_IMAGE_SIZE = 3000000 #this et the maximum number of pixels we authorize for an image
#this has to be lowered in case of GPUOutOfMemory error 


In [11]:
#some useful functions

def list_directory_contents(directory):
    """return a list of all directories and folder inside directory"""
    pictures_list = []

    try:
        with os.scandir(directory) as entries:
            for entry in entries:
                #print(entry.name)
                pictures_list.append(entry.name)
    except FileNotFoundError:
        print(f"The directory {directory} does not exist.")
    except NotADirectoryError:
        print(f"{directory} is not a directory.")
    except PermissionError:
        print(f"Permission denied to access {directory}.")
    return pictures_list

def list_files(startpath,exclude = "Vrac"):
    """From a stratpath this function list all files with full directory
    We allow to exclude one folder (can be easily adapted to several)"""
    file_list = []
    for root, dirs, files in os.walk(startpath):
        if not exclude in root:
            for file in files:
                file_list.append(os.path.join(root, file))
    return file_list



In [8]:
#loading SAM

model_type = "vit_l" #is hace more or less gpu memory, you can switch to less or more heavy models, see http://github.com/facebookresearch/segment-anything for more details and checkpoint download
sam = sam_model_registry[model_type](checkpoint = '/home/gabriel/Documents/TR DIMA/codes/sam_checkpoint/sam_vit_l_0b3195.pth') #put here the path to checkpoint
sam.cuda() #moving model to cuda

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-23): 24 x Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=1024, out_features=4096, bias=True)
          (lin2): Linear(in_features=4096, out_features=1024, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d

**Section 1 :** 
Image cropping $ \\ $
(you must restart the kernel each time you wan't to lauch the code of this section to avoid GPUoutofmemoryerror)
 $ \\ $
 You may have issues with GPU memory if you lauch the cell with sam.cuda() instruction several time during the same session

In [ ]:
#creating AUtomaticMaskGenerator instance

mask_generator = SamAutomaticMaskGenerator(sam,min_mask_region_area=5000)

#the second parameter stand to avoid masks with several connected components with one of them to small to be a wing
#he might be ajusted if your dataset have very high quality or very low quality


In [ ]:
#useful functions for section 1

def Find_bbox(mask):
    """return x_min,y_min,x_max,y_max for the mask"""
    mask = np.array(mask,dtype = np.uint8)
    mask_coords = np.argwhere(mask == 1)
    xmin,ymin,xmax,ymax = np.min(mask_coords[:,0]),np.min(mask_coords[:,1]),np.max(mask_coords[:,0]),np.max(mask_coords[:,1])
    return xmin,ymin,xmax,ymax

def Rotation_matrix(theta):

    return np.array([[np.cos(theta), -np.sin(theta)],
                     [np.sin(theta), np.cos(theta)]])


def Find_Optimal_Rotation(mask):

    mask = np.array(mask,dtype = np.uint8)

    where_array = np.argwhere(mask == 1)

    def objective_function(theta):
        R_m = Rotation_matrix(theta)
        rotated_mask_coordinates = np.dot(where_array,R_m)
        xmin,ymin,xmax,ymax = np.min(rotated_mask_coordinates[:,0]),np.min(rotated_mask_coordinates[:,1]),np.max(rotated_mask_coordinates[:,0]),np.max(rotated_mask_coordinates[:,1])
        width = ymax - ymin
        height = xmax - xmin
        return -width/height
    
    result = minimize(objective_function,x0 = 0.0,method='BFGS',tol = 0.0001)
    best_angle = np.squeeze(result.x,axis = 0)
    best_ratio = np.abs(result.fun)
    return best_angle,best_ratio

def Universal_crop_reshaper(wing_picture,wing_mask,goal_shape = (256,512)):
    """return the reshape crop of wing with boundarie adjustment to fit goal_shape
     without any deformations """
    xmin_i,ymin_i,xmax_i,ymax_i = Find_bbox(wing_mask)
    x,y = goal_shape  

    ratio_ini = (ymax_i-ymin_i)/(xmax_i-xmin_i)
    ratio_f = y/x


    if ratio_f > ratio_ini:
        ymin_f = int(max(0,(ymax_i+ymin_i)/2-ratio_f/ratio_ini*(ymax_i-ymin_i)/2))
        ymax_f = int(ymin_f + (ymax_i-ymin_i)*ratio_f/ratio_ini)
        xmin_f,xmax_f  = xmin_i,xmax_i
    else:
        xmin_f = int(max(0,(xmax_i+xmin_i)/2-ratio_ini/ratio_f*(xmax_i-xmin_i)/2))
        xmax_f = int(xmin_f + (xmax_i-xmin_i)*ratio_ini/ratio_f)
        ymin_f,ymax_f  = ymin_i,ymax_i

    cropped_wing = wing_picture[xmin_f:xmax_f,ymin_f:ymax_f]
    resized_cropped_wing = cv2.resize(cropped_wing,(y,x))
    return resized_cropped_wing

def segment_and_crop(wing_image,option = 'C'):
    """this function take image with wing and white background
      as argument and return segmented and cropped wing as np array
      """
    gc.collect()
    torch.cuda.empty_cache()  #clear gpu memory (don' t know if it's really useful)
    
    xi,yi,zi = wing_image.shape
    reduction_factor = max(1,(xi*yi/MAX_IMAGE_SIZE)**(1/2)) 
    #the parameter above is necessary, because some images are too big, and this cause issues for SAM inference
    image_r = cv2.resize(wing_image, (int(yi/reduction_factor), int(xi/reduction_factor)))

    # Generate masks
    with torch.no_grad():
        masks = mask_generator.generate(image_r) #generating masks

    current_max_area = 0   #this will store the current area of the object that verify the condition below : the goal is to extract the largest one that respect that condition
    wing_mask = None
    effective_best_rotation = 0 #in case something went wrong, we set default best rotation to 0
    effective_best_ratio = None

    #thanks to the parameters set for AutomaticMaskGenerator, we don't need a filter for largest connected component
    #we however have to compute the rotation that maximize width/height factor, to use this to discriminate masks
    #this will be then used to rotate wings
    


    for mask in masks:
        effective_mask = np.array(mask['segmentation'],dtype = np.uint8)
        best_rotation,best_ratio = Find_Optimal_Rotation(effective_mask)

        if 2.2<best_ratio<2.8: #parameters set empirically
            if mask['area'] > current_max_area:
                effective_best_rotation = best_rotation
                effective_best_ratio = best_ratio #this variable is never used
                wing_mask = mask
                current_max_area = mask['area'] #update current_max_area value


    effective_best_rotation = effective_best_rotation  * 180/np.pi #conversion to degree
    effective_mask = rotate(np.array(wing_mask['segmentation'],dtype = np.float32),angle=effective_best_rotation,reshape = True)
    effective_mask = np.round(effective_mask).astype(np.uint8)
   




    wing_image_rotated = rotate(image_r,angle=effective_best_rotation,reshape = True)

    if option == 'C':
        xmin,ymin,xmax,ymax = Find_bbox(effective_mask)
        cropped_rotated_wing = wing_image_rotated[xmin:xmax,ymin:ymax]
        cropped_rotated_wing = Universal_crop_reshaper(wing_image_rotated,effective_mask)
    elif option == 'S':
        xmin,ymin,xmax,ymax = Find_bbox(effective_mask)
        effective_mask = np.expand_dims(effective_mask,axis = 2).astype(np.float32)
        cropped_rotated_wing = wing_image_rotated[xmin:xmax,ymin:ymax]*effective_mask[xmin:xmax,ymin:ymax]


    return cropped_rotated_wing






In [ ]:
#opening and processing images
#run this cell if you want to test the above protocol "on the fly", or test it on specific images

wing_images_directory = "/home/gabriel/Documents/TR DIMA/data/Images_orga_cropping"

image_names = list_files(wing_images_directory)
test_name = "/home/gabriel/Documents/TR DIMA/data/Images_orga_cropping/Bombus_ruderarius/Ouvriere/ABAURA4902/ABAURA4902_S3.jpg"

images_test = image_names[900:905] #you can for instance take some images in your images folder

for dir in images_test:
    current_image = cv2.imread(dir)
    current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2RGB)

    current_crop = segment_and_crop(current_image,option='C')

    plt.imshow(current_image)
    plt.axis('off')
    plt.show()

    plt.imshow(current_crop)
    plt.show()

In [ ]:
#cropping images and saving them into a specific folder

wing_images_directory = "/home/gabriel/Documents/TR DIMA/data/Images_orga_cropping"

list_file_names = list_files(wing_images_directory)